# GNS Implementation Details

The model works by adopting a particle-based representation of the physical system. Physical dynamics are approximated by interactions among the particles. The objective of the model is to learn these interactions.

## Input and output representations
### Inputs
Each particle's input state vector represents:
- Position, $p_i^{t}$
- A sequence of $C=5$ previous velocities. The velocity is calculated from the difference in position between consecutive time steps: $\dot{p}^t=p^t-p^{t-1}$
- Features that capture the static material properties (e.g. water, sand, rigid, etc..). The material is expressed as a particle feature, $a_i$, represented with a learned embedding vector of size 16.
- The global properties of the system, $g$, include external forces and global material properties.
- For datasets with fixed flat orthogonal walls, instead of adding boundary particles, a feature is added to each node indicating the vector distance to each wall, $d^{t}_i$. To maintain spatial transalation invariance, this distance is clipped to the connectivity radius $R$, achieving a similar effect to that of the boundary particles.

The particle feature tensor looks as follows:
$$x^{t}_i = [p^{t}_i,\dot{p}^{t-C+1}_i,...,\dot{p}^{t}_i,a_i, g, d^{t}_i]$$


### Outputs
The prediction targets for supervised learning are the per-particle average acceleration, $$\ddot{p}^t_i=\dot{p}^{t+1}-\dot{p}^t=p^{t+1}-2p^{t}+p^{t-1}$$


## Encoder
The encoder embeds the particle-based state representation, $X$, as a latent graph $G_0=\text{ENCODER}(X)$, where $G=(V,E,\mathbf{u})$, $\mathbf{v}_i\in V$, and $\mathbf{e}_{i,j}\in E$.
- The encoder constructs the graph structure $G^0$ by assignning a node to each particle and adding edges between particles within a connectivity radius, $R$. On each timestep the graph's edges are recomputed by a nearest neighbor algorithm, implemented by a standard kd-tree, to reflect the current particle positions.
- The node embeddings, $\mathbf{v}_i=\varepsilon ^v(x_i)$, are learned functions of the particles' states.
- The edge embeddings, $\mathbf{e}_{i,j}=\varepsilon^e(\mathbf{r}_{i,j})$, are learned functions of the pairwise properties of the corresponding particles, $\mathbf{r}_{i,j}$, e.g., displacement between their positions, spring constant, etc.
- $\varepsilon^v$ and $\varepsilon^e$ as a multilayer perceptron, which encode node features and edge features into the latent vectors, $v_i$ and $e_{i,j}$, of size $128$.
- The graph-level embedding, $\mathbf{u}$, could represent global properties such as gravity and magnetic fields. Although, this is currently implemented as node level features instead.

## Processor
...

## Decoder
...

## Noise
...TODO...

## Normalization
...TODO...

## Loss function



In [ ]:
# Dataset
import numpy as np
import h5py
import torch
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.transforms import RadiusGraph

R=0.08 # Connectivity radius $R$

class GNSDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(GNSDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return ['box_bath.hdf5']
    
    @property
    def processed_file_names(self):
        return ['box_bath_samples.pt']
    
    def process(self):
        # TODO: Split into train, test, val
        data_list = []
        # Read all positions & transform into features
        f = h5py.File(self.raw_file_names[0],'r')
        for k in range(1): # TODO: f.get('rollouts').keys():
            # Read positions
            positions = np.array(f.get(f'rollouts/{k}/positions'))
            # Calculate velocities
            velocities = np.concatenate(([np.zeros(positions[0].shape)],
                                        positions[1:] - positions[0:-1]),axis=0)
            # Calculate accelerations
            accelerations = np.concatenate(([np.zeros(velocities[0].shape)],
                                        velocities[1:] - velocities[0:-1]),axis=0)
            # Material properties (using one-hot encoding for now)
            m = np.zeros((len(positions[0]), 2))
            m[0:64] = [0,1] # First 64 particles are solid
            m[64:] = [1,0]
            # TODO: Global forces
            # TODO: Distance to walls
            x = []
            y = accelerations[6:-1]
            # Drop the first 5 and the last step since we don't have accurate velocities/accelerations
            for t in range(6,len(positions)-1):
                xt = np.concatenate((positions[t], m, np.concatenate(velocities[t-5:t], axis=1)), axis=1)
                x.append(xt)
            data = Data(x=torch.Tensor(x), y=torch.Tensor(y), pos=torch.Tensor(positions[6:-1]))
            # Apply pre-transform to get edges
            calculate_edges = self.pre_transform or RadiusGraph(R)
            data = calculate_edges(data)
            data_list.append(data)
        torch.save(self.collate(data_list), 'box_bath_samples.pt')
    
dataset = GNSDataset('.')
print(len(dataset), dataset.num_classes, dataset.num_node_features)
print(dataset[0])


# Parameters
Parameters for BoxBath:
- Trajectory length: 150
- Number of rollouts: Train/Validation/Test -> 2700/150/150
- Connectivity radius: $R=0.08$